# libs

In [19]:
import midi
import numpy as np
import pandas as pd
import random
import uuid

# https://code.google.com/archive/p/python-musical/source/default/source
# pretty awesome library. I downloaded the package from the above link and then extracted
# the musical/ directory from python-musical/trunk/ and placed in sister midi-skirt/ (.gitignored). 
from musical.theory import Note, scale, Scale, Chord

# constants

In [20]:
class PatternConstants:
    def __init__(self, resolution=440):
        self.resolution = resolution
        self.quarter_note = resolution
        self.half_note = resolution * 2
        self.eighth_note = int(resolution / 2.0)
        self.sixteenth_note = int(resolution / 4.0)
        self.thirty_second_note = int(resolution / 8.0)
        self.sixty_forth_note = int(resolution / 16.0)
        self.whole_note = resolution * 4
        self.bar = resolution * 4

bpm = 160

pc = PatternConstants()

# pattern and track

In [21]:
pattern = midi.Pattern(resolution=pc.resolution)
rhythm_track = midi.Track()
melody_track = midi.Track()
# tempo = midi.SetTempoEvent(bpm=120)
# track.append(tempo)
pattern.append(rhythm_track)
pattern.append(melody_track)

# helper flunctions

In [23]:
def find_nearest_note(value, note_type):
    mod = value % note_type
    if (mod > (note_type / 2.0)): # round up 
        return value + note_type - mod
    else:
        return value - mod

In [24]:
def make_ticks_rel(track):
    number_before_negative = 0
    running_tick = 0
    for event in track:
        event.tick -= running_tick
        if event.tick >= 0:
            number_before_negative += 1
        else:
            print number_before_negative
        running_tick += event.tick
    return track

In [25]:
def get_max_tick(track):
    return max([event.tick for event in track])

def get_min_tick(track):
    return min([event.tick for event in track])

In [ ]:
def add_tuples_to_track(track, tuples):
    all_note_events = pd.DataFrame(tuples)
    all_note_events.columns = ["note_id", "event_type_fun", "tick", "pitch", "note_length", "velocity"]
    all_note_events.sort_values(by=["tick", "note_length"], inplace=True)
    track.append(midi.InstrumentNameEvent(tick=0, text='Modern Classic', data=[]))
    for row in all_note_events.iterrows():
        data = row[1]
        track.append(data["event_type_fun"](tick=data["tick"], velocity=data["velocity"], pitch=data["pitch"]))
    return track

# scale specs

In [26]:
scale.NAMED_SCALES

{'aeolian': (2, 1, 2, 2, 1, 2, 2),
 'augmented': (3, 1, 3, 1, 3, 1),
 'augmentedfifth': (2, 2, 1, 2, 1, 1, 2, 1),
 'bluesmajor': (3, 2, 1, 1, 2, 3),
 'bluesminor': (3, 2, 1, 1, 3, 2),
 'chromatic': (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1),
 'diminished': (2, 1, 2, 1, 2, 1, 2, 1),
 'dorian': (2, 1, 2, 2, 2, 1, 2),
 'halfwhole': (1, 2, 1, 2, 1, 2, 1, 2),
 'harmonicminor': (2, 1, 2, 2, 1, 3, 1),
 'ionian': (2, 2, 1, 2, 2, 2, 1),
 'japanese': (1, 4, 2, 1, 4),
 'locrian': (1, 2, 2, 1, 2, 2, 2),
 'lydian': (2, 2, 2, 1, 2, 2, 1),
 'major': (2, 2, 1, 2, 2, 2, 1),
 'melodicminor': (2, 1, 2, 2, 2, 2, 1),
 'minor': (2, 1, 2, 2, 1, 2, 2),
 'mixolydian': (2, 2, 1, 2, 2, 1, 2),
 'oriental': (1, 3, 1, 1, 3, 1, 2),
 'pentatonicmajor': (2, 2, 3, 2, 3),
 'pentatonicminor': (3, 2, 2, 3, 2),
 'phrygian': (1, 2, 2, 2, 1, 2, 2),
 'wholehalf': (2, 1, 2, 1, 2, 1, 2, 1),
 'wholetone': (2, 2, 2, 2, 2, 2)}

In [27]:
def get_list_of_chord_notes_from_chord(chord):
    return [Note.index_from_string(chord_note.note + str(chord_note.octave)) for chord_note in my_chord]

In [28]:
root = Note(('C', 0))
named_scale = scale.NAMED_SCALES['major']
my_scale = Scale(root, named_scale)
chords_in_scale = Chord.progression(my_scale, 5)
# my_chords = chords_in_scale[0:4]
my_chords = [chords_in_scale[0]] + [chords_in_scale[5]] + [chords_in_scale[3]] + [chords_in_scale[4]]
#my_chord = Chord.fromscale(root, my_scale)
chord_notes = [get_list_of_chord_notes_from_chord(my_chord) for my_chord in my_chords]

In [29]:
chords_in_scale

[Chord((Note('c5'), Note('e5'), Note('g5'))),
 Chord((Note('d5'), Note('f5'), Note('a5'))),
 Chord((Note('e5'), Note('g5'), Note('b5'))),
 Chord((Note('f5'), Note('a5'), Note('c6'))),
 Chord((Note('g5'), Note('b5'), Note('d6'))),
 Chord((Note('a5'), Note('c6'), Note('e6'))),
 Chord((Note('b5'), Note('d6'), Note('f6')))]

In [30]:
chord_notes

[[60, 64, 67], [69, 72, 76], [65, 69, 72], [67, 71, 74]]

# chord rhythm
really this is just a melody but with chords. eventually I should just use Melody. Rhythm should be agnostic to notes or scale

In [31]:
class Rhythm:
    def __init__(self, rhythm_len=None, start_tick=None, quantization=None, chord_density=None, note_len_choices=None,
                chord_notes=None):
        self.rhythm_len = rhythm_len
        self.start_tick = start_tick
        self.quantization = quantization  # this maybe should be at note level only
        self.chord_density = chord_density  # proportion of available ticks (determined by quantization) occupied by chords
        self.note_len_choices = note_len_choices
        self.chord_notes = chord_notes
        self.number_of_chords = self._compute_num_chords()
        self.start_ticks = self._get_start_ticks()
    
    def _compute_num_chords(self):
        return self.rhythm_len / self.quantization
    
    def _get_start_ticks(self):
        return np.unique(sorted([find_nearest_note(random.randint(0, self.rhythm_len), self.quantization)
                                 for _ in range(self.number_of_chords)]))

    def create_rhythm(self):
        rhythm = []
        for tick in self.start_ticks:
            tick += self.start_tick
            chord_tuples = self._create_note_tuples_from_chord(tick, self.chord_notes)
            rhythm.extend(chord_tuples)
        return rhythm
    
    def _create_note_tuples_from_chord(self, start_tick, notes):
        # returns a list of 2*len(notes) tuples: [(note_id, event_type, tick, note, velocity)]
        ret = []
        note_length = random.choice(self.note_len_choices)
        for note in notes:
            notes_created = self._create_note_tuple(start_tick, note, note_length, random.randint(50, 90))
            ret.extend(notes_created)
        return ret

    def _create_note_tuple(self, start_tick, note, note_length, velocity):
        # returns a list of two tuples: (note_id, event_type, tick, note, velocity)
        note_id = str(uuid.uuid1())
        return [
            (note_id, midi.NoteOnEvent, start_tick, note, note_length, velocity),
            (note_id, midi.NoteOffEvent, start_tick+note_length, note, note_length, 0)
        ]

In [32]:
# hard-coded in melody: velocity of notes, octave
rhythm = Rhythm(
    rhythm_len=pc.bar*64,
    start_tick=0,
    quantization=pc.eighth_note,
    chord_density=.1,
    note_len_choices=[pc.quarter_note, pc.eighth_note, pc.sixteenth_note, pc.thirty_second_note,
                      pc.half_note, pc.sixty_forth_note],
    chord_notes=None)

# my_rhythm = rhythm.create_rhythm()

# chord progression specs

In [33]:
mel_len = 64 * pc.bar
changes = np.arange(0, mel_len, 2*c.bar)
chords = chord_notes * 8

In [34]:
class ChordProgression:
    def __init__(self, chords=None, start_ticks=None):
        self.chords = chords
        self.start_ticks = start_ticks
    
    def create_rhythms(self):
        all_rhythms = []
        for i in range(len(self.chords)):
            all_rhythms.extend(self.create_rhythm(
                    start_tick=self.start_ticks[i],
                    chord_notes=self.chords[i]))
        return all_rhythms
    
    def create_rhythm(self, start_tick, chord_notes):
        rhythm = Rhythm(
            rhythm_len=np.diff(self.start_ticks)[0],
            start_tick=start_tick,
            quantization=pc.eighth_note,
            chord_density=.1,
            note_len_choices=[pc.quarter_note, pc.eighth_note, pc.sixteenth_note, pc.thirty_second_note,
                              pc.half_note, pc.sixty_forth_note],
            chord_notes=chord_notes)

        return rhythm.create_rhythm()

In [35]:
cp = ChordProgression(chords, changes)

In [36]:
my_rhythm = cp.create_rhythms()

# melody

In [37]:
class Melody:
    def __init__(self, melody_len=None, scale=None, quantization=None, note_density=None, note_len_choices=None,
                 available_notes=None):
        self.melody_len = melody_len
        self.quantization = quantization  # this maybe should be at note level only
        self.note_density = note_density  # proportion of available ticks (determined by quantization) occupied by notes
        self.note_len_choices = note_len_choices
        self.available_notes = available_notes
        self.number_of_notes = self._compute_num_notes()
        self.start_ticks = self._get_start_ticks()
    
    def _compute_num_notes(self):
        return self.melody_len / self.quantization
    
    def _get_start_ticks(self):
        return np.unique(sorted([find_nearest_note(random.randint(0, self.melody_len), self.quantization)
                                 for _ in range(self.number_of_notes)]))

    def create_melody(self):
        melody_notes = []
        for tick in self.start_ticks:
            melody_tuples = self._create_melody_note_tuple(tick)
            melody_notes.extend(melody_tuples)
        return melody_notes
    
    def _create_melody_note_tuple(self, start_tick):
        note_id = str(uuid.uuid1())
        velocity = random.randint(50, 90)
        cur_note = random.choice(self.available_notes)
        cur_note = Note.index_from_string(cur_note.note + str(cur_note.octave))
        note_length = random.choice(self.note_len_choices)
        return [
            (note_id, midi.NoteOnEvent, start_tick, cur_note, note_length, velocity),
            (note_id, midi.NoteOffEvent, start_tick+note_length, cur_note, note_length, 0)
        ]

In [38]:
# note2self: hard-coded in melody: velocity of notes, octave
melody = Melody(
    melody_len=pc.bar*64,
    quantization=pc.sixteenth_note,
    note_density=.4,
    note_len_choices=[pc.quarter_note, pc.eighth_note, pc.sixteenth_note, pc.sixty_forth_note,
                      pc.half_note, pc.thirty_second_note],
    available_notes=[my_scale.get(x) for x in range(28, 37)])

my_melody = melody.create_melody()

In [40]:
rhythm_track = add_tuples_to_track(rhythm_track, my_rhythm)
melody_track = add_tuples_to_track(melody_track, my_melody)

# end of track

In [41]:
# Add the end of track event, append it to the track
eot = midi.EndOfTrackEvent(tick=get_max_tick(rhythm_track) + 2*pc.whole_note)  # probably an excessive buffer
rhythm_track.append(eot)

eot = midi.EndOfTrackEvent(tick=get_max_tick(melody_track) + 2*pc.whole_note)
melody_track.append(eot)

In [42]:
rhythm_track = make_ticks_rel(rhythm_track)
melody_track = make_ticks_rel(melody_track)

# write to file

In [43]:
midi.write_midifile("example.mid", pattern)